In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv(r"C:\Users\rajat.chauhan\Downloads\Data science\Youtube Comments Analysis\processed.csv")
df

,Unnamed: 0,clean_comment,category
0,0,family mormon never tried explain still stare ...,1
1,1,buddhism much lot compatible christianity espe...,1
2,2,seriously say thing first get complex explain ...,-1
3,3,learned want teach different focus goal not wr...,0
4,4,benefit may want read living buddha living chr...,1
...,...,...,...
36241,37243,agree push make nation either pity pakistan in...,-1
36242,37244,jesus,0
36243,37245,kya bhai pure saal chutiya banaya modi aur jab...,1
36244,37246,downvote karna tha par upvote hogaya,0


In [4]:
df.dropna(inplace=True)

In [5]:
import dagshub
dagshub.init(repo_owner='rajatchauhan99', repo_name='Youtube-Comment-Analysis-Chrome-Plugin', mlflow=True)

Accessing as rajatchauhan99

Initialized MLflow to track repo "rajatchauhan99/Youtube-Comment-Analysis-Chrome-Plugin"

Repository rajatchauhan99/Youtube-Comment-Analysis-Chrome-Plugin initialized!

In [6]:
# Set or create an experiment
import mlflow
mlflow.set_experiment("Handling data leakage Exp 5 - lightgbm with hpt")

2024/11/05 18:37:20 INFO mlflow.tracking.fluent: Experiment with name 'Handling data leakage Exp 5 - lightgbm with hpt' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/e3fadd3c574b41e79cd00bc79ffb70d6', creation_time=1730812040272, experiment_id='8', last_update_time=1730812040272, lifecycle_stage='active', name='Handling data leakage Exp 5 - lightgbm with hpt', tags={}>

In [7]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

c:\Users\rajat.chauhan\conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Here's the revised sequence to optimize the workflow and avoid data leakage:

1. **Train-Test Split:** Perform the split before applying TF-IDF and SMOTE. This ensures that the test set remains untouched by feature extraction and synthetic sampling, preserving its integrity.

2. **Label Mapping and NaN Removal:** These steps should be performed on the entire dataset to prepare the labels before splitting into training and testing sets.

3. **TF-IDF Vectorization on Training Set Only:** Fit the `TfidfVectorizer` on the training set, then transform both the training and test sets.

4. **SMOTE on Training Data Only:** Apply SMOTE after vectorization to handle class imbalance, but only on the training data to avoid leaking synthetic samples into the test set.

5. **Train the Model with MLflow and Optuna:** Log results, parameters, and models.

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, classification_report
import mlflow
import optuna
from lightgbm import LGBMClassifier

# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: Train-test split
X_train_text, X_test_text, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

# Step 4: TF-IDF Vectorizer setup and fit on training data
ngram_range = (1, 3)  # Trigram
max_features = 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

# Step 5: Apply SMOTE to the training data only
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for LightGBM
def objective_lightgbm(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = LGBMClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    return accuracy_score(y_test, model.fit(X_train_resampled, y_train_resampled).predict(X_test))


# Step 7: Run Optuna for LightGBM, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_lightgbm, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = LGBMClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "LightGBM"
    log_mlflow("LightGBM", best_model, X_train_resampled, X_test, y_train_resampled, y_test)

# Run the experiment for LightGBM
run_optuna_experiment()

[I 2024-11-05 18:41:20,472] A new study created in memory with name: no-name-6f71b08a-2d95-4141-9b67-75df45adbe67


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.029753 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 99663
[LightGBM] [Info] Number of data points in the train set: 37599, number of used features: 977
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


[I 2024-11-05 18:41:22,556] Trial 0 finished with value: 0.5606290522830736 and parameters: {'n_estimators': 90, 'learning_rate': 0.00010561115673818157, 'max_depth': 7}. Best is trial 0 with value: 0.5606290522830736.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030988 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 99663
[LightGBM] [Info] Number of data points in the train set: 37599, number of used features: 977
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

[I 2024-11-05 18:41:25,326] Trial 1 finished with value: 0.5308318388743275 and parameters: {'n_estimators': 179, 'learning_rate': 0.0002285009192434133, 'max_depth': 5}. Best is trial 0 with value: 0.5606290522830736.


...

[I 2024-11-05 18:43:12,634] Trial 28 finished with value: 0.6028417712787971 and parameters: {'n_estimators': 272, 'learning_rate': 0.00041856744150534195, 'max_depth': 9}. Best is trial 25 with value: 0.785487653469444.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.032893 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 99663
[LightGBM] [Info] Number of data points in the train set: 37599, number of used features: 977
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


[I 2024-11-05 18:43:16,743] Trial 29 finished with value: 0.6880949096427094 and parameters: {'n_estimators': 190, 'learning_rate': 0.013059400262829002, 'max_depth': 7}. Best is trial 25 with value: 0.785487653469444.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.028457 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 99663
[LightGBM] [Info] Number of data points in the train set: 37599, number of used features: 977
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

2024/11/05 18:43:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/05 18:43:56 INFO mlflow.tracking._tracking_service.client: 🏃 View run LightGBM_SMOTE_TFIDF_Trigrams at: https://dagshub.com/rajatchauhan99/Youtube-Comment-Analysis-Chrome-Plugin.mlflow/#/experiments/8/runs/cbcf84e7080e44db9d72d66b849904dd.
2024/11/05 18:43:56 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/rajatchauhan99/Youtube-Comment-Analysis-Chrome-Plugin.mlflow/#/experiments/8.



### Key Changes
1. **Train-Test Split** before TF-IDF and SMOTE.
2. **TF-IDF Vectorization** fitted on training data and transformed on both train and test sets.
3. **SMOTE applied only on the training data** after vectorization. 

This sequence prevents data leakage and allows the model to generalize better on unseen data.